In [1]:
import numpy as np
import quantities as pq

from elephant.spike_train_generation import homogeneous_poisson_process
from elephant.statistics import mean_firing_rate, time_histogram

from elephant.parallel import SingleProcess, ProcessPoolExecutor
from elephant.conversion import BinnedSpikeTrain
from elephant.spike_train_correlation import correlation_coefficient
from elephant.statistics import isi, lv, cv2, lvr

import mpi4py
from elephant.parallel.mpi import MPIPoolExecutor, MPICommExecutor
import networkunit as nu

NameError: name 'use_cache' is not defined

In [8]:
import time

def wait(spiketrain):
    print(0.1)
    time.sleep(0.1)
    return None

In [2]:
rate = 10 * pq.Hz
spiketrains = [[homogeneous_poisson_process(rate, t_stop=10*pq.s) for x in range(10)] for _ in range(10)]

In [7]:
model = nu.models.stochastic_activity()
model.spiketrains = spiketrains[0]

# test = nu.tests.average_correlation_test()
# test = nu.tests.firing_rate_test()
test = nu.tests.isi_variation_test(variation_measure='isi')
pred = test.generate_prediction(model)
pred.units

array(1.) * dimensionless

In [17]:
def calc_avg_correlations(spiketrains):
    if len(spiketrains) == 1:
        avg_corr = np.array([np.nan])
    else:
        cc_matrix = self.generate_cc_matrix(spiketrains)
        np.fill_diagonal(cc_matrix, np.nan)

        avg_corr = np.nansum(cc_matrix, axis=0) / len(spiketrains)
    return avg_corr

10
(2000, 1)


In [8]:
%%timeit
firing_rate0 = SingleProcess().execute(wait, spiketrains)

1 s ± 167 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
firing_rate1 = ProcessPoolExecutor().execute(wait, spiketrains)

254 ms ± 3.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
firing_rate2 = MPIPoolExecutor().execute(wait, spiketrains)

[PC0W9KFR][[1112,1],0][btl_tcp_endpoint.c:730:mca_btl_tcp_endpoint_start_connect] Exception in thread Thread-15:
Traceback (most recent call last):
  File "/home/rgutzen/anaconda3/envs/py38/lib/python3.9/site-packages/mpi4py/futures/_lib.py", line 376, in barrier
bind on local address (172.18.119.17:0) failed: Cannot assign requested address (99)
Error in MPI_Isend(0, 0, 0x7fe3e5c011e0, 0, -27, 94709414371408) (-1)
[PC0W9KFR][[1112,1],0][btl_tcp_endpoint.c:730:mca_btl_tcp_endpoint_start_connect] bind on local address (172.18.119.17:0) failed: Cannot assign requested address (99)
    request = comm.Ibarrier()
  File "mpi4py/MPI/Comm.pyx", line 747, in mpi4py.MPI.Comm.Ibarrier
mpi4py.MPI.Exception: MPI_ERR_OTHER: known error not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rgutzen/anaconda3/envs/py38/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/rgutzen/anaconda3/en

KeyboardInterrupt: 

In [ ]:
%%timeit
firing_rate3 = MPICommExecutor().execute(wait, spiketrains)

In [ ]:
print(MPIPoolExecutor())

In [8]:
class parallelize:
    """
    Context manager that applies elephant.parallel executors:
    ProcessPoolExecutor(), MPIPoolExecutor(), MPICommExecutor(), or
    SingleProcess() (default).

    Example:
    ```
    results = [my_function(arg) for arg in iterables_list]
    ```
    becomes
    ```
    with parallelize(my_function) as parallel_func:
        results = parallel_func(iterables_list, **kwargs)
    ```
    """
    def __init__(self, func, test_class=None, executor=SingleProcess()):
        self.executor = executor
        self.func = func

        if test_class is not None and'parallel_executor' in test_class.params:
            self.executor = test_class.params['parallel_executor']

    def __enter__(self):

        def _func(iterable_list, **kwargs):

            def arg_func(iterable_list):
                return self.func(iterable_list, **kwargs)

            result = self.executor.execute(arg_func, iterable_list)
            return result

        return _func

    def __exit__(self, exc_type, exc_value, exc_traceback):
        pass

In [10]:
def parallel_stuff(iterable):
    i, (a, b) = iterable
    print(i, a, b)
    
with parallelize(parallel_stuff) as func:
    func(enumerate(zip(range(3), range(3,6))))

0 0 3
1 1 4
2 2 5
